# Weave


![Wandb Logo](http://wandb.me/logo-im-png)


[Weave](https://weave-docs.wandb.ai/) is designed to make tracking and logging all calls made through the [LangChain Python library](https://github.com/langchain-ai/langchain) effortless.

When working with LLMs, debugging is inevitable. Whether a model call fails, an output is misformatted, or nested model calls create confusion, pinpointing issues can be challenging. LangChain applications often consist of multiple steps and LLM call invocations, making it crucial to understand the inner workings of your chains and agents.

Weave simplifies this process by automatically capturing traces for your [LangChain](https://python.langchain.com/v0.2/docs/introduction/) applications. This enables you to monitor and analyze your application's performance, making it easier to debug and optimize your LLM workflows.


## Getting Started

### Installation

To install the Weave SDK, run the following command in your terminal:

```bash
pip install -U weave
```
 or alternatively, install it in your notebook as follows:

```python
!pip install -U weave
```


To get started, simply call `weave.init()` at the beginning of your script. The argument in `weave.init()` is a project name that will help you organize your traces.

If this is your first time using W&B or you are not logged in, the link that appears after running `weave.init` will take you to sign-up/login page. Signing up is easy!


In [ ]:
import weave
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize Weave with your project name
weave.init("langchain_demo")

llm = ChatOpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

llm_chain = prompt | llm

output = await llm_chain.ainvoke({"number": 2})

print(output)

## Traces

Storing traces of LLM applications in a central database is crucial during both development and production. These traces are essential for debugging and improving your application by providing a valuable dataset.

Weave automatically captures traces for your LangChain applications. It will track and log all calls made through the LangChain, including prompt templates, chains, LLM calls, tools, and agent steps. You can view the traces in the Weave web interface.

[![langchain_trace.png](https://weave-docs.wandb.ai/assets/images/langchain_trace-8a5aac7070df8635840cfb6fbb43996f.png)](https://wandb.ai/parambharat/langchain_demo/weave/calls)


## Manually Tracing Calls

In addition to automatic tracing, you can manually trace calls using the `WeaveTracer` callback or the `weave_tracing_enabled` context manager. These methods are akin to using request callbacks in individual parts of a LangChain application.

**Note:** Weave traces Langchain Runnables by default and this is enabled when you call `weave.init()`. You can disable this behaviour by setting the environment variable `WEAVE_TRACE_LANGCHAIN` to `"false"` before calling `weave.init()`. This allows you to control the tracing behaviour of specific chains or even individual requests in your application.



### Using `WeaveTracer`

You can pass the `WeaveTracer` callback to individual LangChain components to trace specific requests.


In [ ]:
import os

os.environ["WEAVE_TRACE_LANGCHAIN"] = "false"  # <- explicitly disable global tracing.

import weave
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from weave.integrations.langchain import WeaveTracer

# Initialize Weave with your project name
weave.init(
    "langchain_demo"
)  # <-- we don't enable tracing here because the env var is explicitly set to `false`

weave_tracer = WeaveTracer()

config = {"callbacks": [weave_tracer]}

llm = ChatOpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

llm_chain = prompt | llm

output = llm_chain.invoke(
    {"number": 2}, config=config
)  # <-- this enables tracing only for this chain invoke.

llm_chain.invoke(
    {"number": 4}
)  # <-- this will not have tracing enabled for langchain calls but openai calls will still be traced


### Using `weave_tracing_enabled` Context Manager

Alternatively, you can use the `weave_tracing_enabled` context manager to enable tracing for specific blocks of code.


In [ ]:
import os

os.environ["WEAVE_TRACE_LANGCHAIN"] = "false"  # <- explicitly disable global tracing.

import weave
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from weave.integrations.langchain import weave_tracing_enabled

# Initialize Weave with your project name
# highlight-next-line
weave.init(
    "langchain_demo"
)  # <-- we don't enable tracing here because the env var is explicitly set to `false`

llm = ChatOpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

llm_chain = prompt | llm

# highlight-next-line
with weave_tracing_enabled():  # <-- this enables tracing only for this chain invoke.
    output = llm_chain.invoke({"number": 2})


llm_chain.invoke(
    {"number": 4}
)  # <-- this will not have tracing enabled for langchain calls but openai calls will still be traced


## Configuration

Upon calling `weave.init`, tracing is enabled by setting the environment variable `WEAVE_TRACE_LANGCHAIN` to `"true"`. This allows Weave to automatically capture traces for your LangChain applications. If you wish to disable this behavior, set the environment variable to `"false"`.

## Relation to LangChain Callbacks

### Auto Logging

The automatic logging provided by `weave.init()` is similar to passing a constructor callback to every component in a LangChain application. This means that all interactions, including prompt templates, chains, LLM calls, tools, and agent steps, are tracked globally across your entire application.

### Manual Logging

The manual logging methods (`WeaveTracer` and `weave_tracing_enabled`) are similar to using request callbacks in individual parts of a LangChain application. These methods provide finer control over which parts of your application are traced:

- **Constructor Callbacks:** Applied to the entire chain or component, logging all interactions consistently.
- **Request Callbacks:** Applied to specific requests, allowing detailed tracing of particular invocations.

By integrating Weave with LangChain, you can ensure comprehensive logging and monitoring of your LLM applications, facilitating easier debugging and performance optimization.

For more detailed information, refer to the [tracing documentation](https://python.langchain.com/v0.2/docs/how_to/debugging/#tracing).

## Models and Evaluations

Organizing and evaluating LLMs in applications for various use cases is challenging with multiple components, such as prompts, model configurations, and inference parameters. Using the [`weave.Model`](https://weave-docs.wandb.ai/guides/core-types/models/), you can capture and organize experimental details like system prompts or the models you use, making it easier to compare different iterations.

The following example demonstrates wrapping a Langchain chain in a `WeaveModel`:


In [ ]:
import json

import weave
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

os.environ["WEAVE_TRACE_LANGCHAIN"] = "True"  # <- explicitly enable global tracing.

# Initialize Weave with your project name
weave.init("langchain_demo")


class ExtractFruitsModel(weave.Model):
    model_name: str
    prompt_template: str

    @weave.op()
    async def predict(self, sentence: str) -> dict:
        llm = ChatOpenAI(model=self.model_name, temperature=0.0)
        prompt = PromptTemplate.from_template(self.prompt_template)

        llm_chain = prompt | llm
        response = llm_chain.invoke({"sentence": sentence})
        result = response.content

        if result is None:
            raise ValueError("No response from model")
        parsed = json.loads(result)
        return parsed


model = ExtractFruitsModel(
    model_name="gpt-3.5-turbo-1106",
    prompt_template='Extract fields ("fruit": <str>, "color": <str>, "flavor": <str>) from the following text, as json: {sentence}',
)
sentence = "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy."

prediction = await model.predict(sentence)

print(prediction)

This code creates a model that can be visualized in the Weave UI:

[![langchain_model.png](https://weave-docs.wandb.ai/assets/images/langchain_model-aa69f5a21ecde2d30be53e3f1b43c601.png)](https://wandb.ai/parambharat/langchain_demo/weave/object-versions?filter=%7B%22baseObjectClass%22%3A%22Model%22%7D&peekPath=%2Fparambharat%2Flangchain_demo%2Fobjects%2FExtractFruitsModel%2Fversions%2FBeoL6WuCH8wgjy6HfmuBMyKzArETg1oAFpYaXZSq1hw%3F%26)


You can also use Weave Models with `serve`, and [`Evaluations`](https://weave-docs.wandb.ai/guides/core-types/evaluations/).



### Evaluations
Evaluations help you measure the performance of your models. By using the [`weave.Evaluation`](/guides/core-types/evaluations) class, you can capture how well your model performs on specific tasks or datasets, making it easier to compare different models and iterations of your application. The following example demonstrates how to evaluate the model we created:



In [ ]:
from weave.flow.scorer import MultiTaskBinaryClassificationF1

sentences = [
    "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy.",
    "Pounits are a bright green color and are more savory than sweet.",
    "Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them.",
]
labels = [
    {"fruit": "neoskizzles", "color": "purple", "flavor": "candy"},
    {"fruit": "pounits", "color": "bright green", "flavor": "savory"},
    {"fruit": "glowls", "color": "pale orange", "flavor": "sour and bitter"},
]
examples = [
    {"id": "0", "sentence": sentences[0], "target": labels[0]},
    {"id": "1", "sentence": sentences[1], "target": labels[1]},
    {"id": "2", "sentence": sentences[2], "target": labels[2]},
]


@weave.op()
def fruit_name_score(target: dict, model_output: dict) -> dict:
    return {"correct": target["fruit"] == model_output["fruit"]}


evaluation = weave.Evaluation(
    dataset=examples,
    scorers=[
        MultiTaskBinaryClassificationF1(class_names=["fruit", "color", "flavor"]),
        fruit_name_score,
    ],
)

scores = await evaluation.evaluate(model)
print(scores)


This code generates an evaluation trace that can be visualized in the Weave UI:

[![langchain_evaluation.png](https://weave-docs.wandb.ai/assets/images/langchain_eval-59b62a719896a5277d9ea397cfb49fd6.png)](https://wandb.ai/parambharat/langchain_demo/weave/calls?filter=%7B%22traceRootsOnly%22%3Atrue%7D&peekPath=%2Fparambharat%2Flangchain_demo%2Fcalls%2F44c3f26c-d9d3-423e-b434-651ea5174be3)

By integrating Weave with Langchain, you can ensure comprehensive logging and monitoring of your LLM applications, facilitating easier debugging and performance optimization.